In [ ]:
!pip install -q sentencepiece
!pip install -q pytorch-lightning
!pip install -q thai2transformers==0.1.2
!pip install -q transformers==4.5.0 sentencepiece
!pip install pythainlp
!pip install nltk
!pip install scikit-learn
!pip install line-bot-sdk
!pip install flask-ngrok
# !pip install torch==1.8.1
# !pip uninstall torch

     |████████████████████████████████| 1.2MB 19.6MB/s 
     |████████████████████████████████| 849kB 17.8MB/s 
     |████████████████████████████████| 829kB 36.3MB/s 
     |████████████████████████████████| 112kB 56.5MB/s 
     |████████████████████████████████| 184kB 54.4MB/s 
     |████████████████████████████████| 276kB 55.2MB/s 
     |████████████████████████████████| 1.3MB 53.3MB/s 
     |████████████████████████████████| 296kB 59.0MB/s 
     |████████████████████████████████| 143kB 59.7MB/s 
     |████████████████████████████████| 1.3MB 18.3MB/s 
     |████████████████████████████████| 1.1MB 53.0MB/s 
     |████████████████████████████████| 133kB 58.9MB/s 
     |████████████████████████████████| 11.0MB 46.8MB/s 
     |████████████████████████████████| 225kB 61.8MB/s 
     |████████████████████████████████| 8.7MB 30.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 10.1MB 46.1MB/s 
     |████████████████████████████████| 2.9MB 48

In [ ]:

#!/usr/bin/python
#-*-coding: utf-8 -*-
##from __future__ import absolute_import
###
from flask import Flask, jsonify, render_template, request
from flask_ngrok import run_with_ngrok
import json
import numpy as np
from google.colab import drive
drive.mount('/content/drive/') 
import sys
sys.path.append('/content/drive/Shareddrives/Pretend To Understand NLP')
import requests
import geopy.distance as ps
import generation_module as gm
# model = gm.load_model()




import delete_module as delete
ngrams = delete.ngrams_counts()
import retrieve_module_tfidf as retrieve
init = retrieve.initialize()

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,TemplateSendMessage,ImageSendMessage, StickerSendMessage, AudioSendMessage, FlexSendMessage
)
from linebot.models.template import *
from linebot import (
    LineBotApi, WebhookHandler
)


app = Flask(__name__)
run_with_ngrok(app) 
model = gm.load_model()
# lineaccesstoken = "LineToken"
line_bot_api = LineBotApi(lineaccesstoken)

####################### new ########################
@app.route('/')
def index():
    return "Hello World!"


@app.route('/webhook', methods=['POST'])
def callback():
    json_line = request.get_json(force=False,cache=False)
    json_line = json.dumps(json_line)
    decoded = json.loads(json_line)
    no_event = len(decoded['events'])
    for i in range(no_event):
        event = decoded['events'][i]
        event_handle(event)
    return '',200


def event_handle(event):
    print(event)
    try:
        userId = event['source']['userId']
    except:
        print('error cannot get userId')
        return ''

    try:
        rtoken = event['replyToken']
    except:
        print('error cannot get rtoken')
        return ''
    if 'message' in event.keys():
        try:
            msgType = event["message"]["type"]
            msgId = event["message"]["id"]
        except:
            print('error cannot get msgID, and msgType')
            sk_id = np.random.randint(1,17)
            replyObj = StickerSendMessage(package_id=str(1),sticker_id=str(sk_id))
            line_bot_api.reply_message(rtoken, replyObj)
            return ''
    if 'postback' in event.keys():
        msgType = 'postback'

    if msgType == "text":
        msg = str(event["message"]["text"])
        delete_output = delete.delete_negative(msg, ngrams.pos_ngrams, ngrams.neg_ngrams)
        if delete_output == 'Pos':
          replyObj = handle_text("พูดเพราะมากค่ะ รักเลยแบบนี้ จุ้ฟ")
          line_bot_api.reply_message(rtoken, replyObj)
        else:
          retrieve_output = retrieve.retrieve_output(delete_output, init.tfidf, init.tfidf_pos, init.pos_attr)
          if retrieve_output == 'curse':
            replyObj = handle_text("ไม่สบถสิจ๊ะคนดี")
            line_bot_api.reply_message(rtoken, replyObj)
          # elif retrieve_output == 'stopwords':
          #   replyObj = handle_text("คิดไม่ออกอะยอมแพ้")
          #   line_bot_api.reply_message(rtoken, replyObj)
          else:
            txt = gm.predict(model,retrieve_output[-1],delete_output[1])
            replyObj = handle_text(str("\n".join(txt)))
            line_bot_api.reply_message(rtoken, replyObj)

    if msgType == "postback":
        msg = str(event["postback"]["data"])
        replyObj = handle_postback(msg)
        line_bot_api.reply_message(rtoken, replyObj)

    if msgType == "location":
        lat = event["message"]["latitude"]
        lng = event["message"]["longitude"]
        #txtresult = handle_location(lat,lng,casedata,3)
        result = getcaseflex(lat,lng)
        replyObj = FlexSendMessage(alt_text='Flex Message alt text', contents=result)
        line_bot_api.reply_message(rtoken, replyObj)
    else:
        sk_id = np.random.randint(1,17)
        replyObj = StickerSendMessage(package_id=str(1),sticker_id=str(sk_id))
        line_bot_api.reply_message(rtoken, replyObj)
    return ''

def getdata(query):
    return 'nodata'

def flexmessage(query):
    res = getdata(query)
    if res == 'nodata':
        return 'nodata'
    else:
        productName,imgUrl,desc,cont = res
    flex = '''
    {
        "type": "bubble",
        "hero": {
          "type": "image",
          "url": "%s",
          "margin": "none",
          "size": "full",
          "aspectRatio": "1:1",
          "aspectMode": "cover",
          "action": {
            "type": "uri",
            "label": "Action",
            "uri": "https://linecorp.com"
          }
        },
        "body": {
          "type": "box",
          "layout": "vertical",
          "spacing": "md",
          "action": {
            "type": "uri",
            "label": "Action",
            "uri": "https://linecorp.com"
          },
          "contents": [
            {
              "type": "text",
              "text": "%s",
              "size": "xl",
              "weight": "bold"
            },
            {
              "type": "text",
              "text": "%s",
              "wrap": true
            }
          ]
        },
        "footer": {
          "type": "box",
          "layout": "vertical",
          "contents": [
            {
              "type": "button",
              "action": {
                "type": "postback",
                "label": "ติดต่อคนขาย",
                "data": "%s"
              },
              "color": "#F67878",
              "style": "primary"
            }
          ]
        }
      }'''%(imgUrl,productName,desc,cont)
    return flex

from linebot.models import (TextSendMessage,FlexSendMessage)
import json

def handle_text(inpmessage):
    flex = flexmessage(inpmessage)
    if flex == 'nodata':
        replyObj = TextSendMessage(text=inpmessage)
    else:
        flex = json.loads(flex)
        replyObj = FlexSendMessage(alt_text='Flex Message alt text', contents=flex)
    return replyObj

def handle_postback(inpmessage):
    replyObj = TextSendMessage(text=inpmessage)
    return replyObj


def handle_location(lat,lng,cdat,topK):
    result = getdistace(lat, lng,cdat)
    result = result.sort_values(by='km')
    result = result.iloc[0:topK]
    txtResult = ''
    for i in range(len(result)):
        kmdistance = '%.1f'%(result.iloc[i]['km'])
        newssource = str(result.iloc[i]['News_Soruce'])
        txtResult = txtResult + 'ห่าง %s กิโลเมตร\n%s\n\n'%(kmdistance,newssource)
    return txtResult[0:-2]


def getcaseflex(lat,lng):
    url = 'https://lavender-chat-bot.herokuapp.com/getnearcase?lat=%s&long=%s'%(lat,lng)
    res = requests.get(url).json()
    return res

def getdistace(latitude, longitude,cdat):
  coords_1 = (float(latitude), float(longitude))
  ## create list of all reference locations from a pandas DataFrame
  latlngList = cdat[['Latitude','Longitude']].values
  ## loop and calculate distance in KM using geopy.distance library and append to distance list
  kmsumList = []
  for latlng in latlngList:
    coords_2 = (float(latlng[0]),float(latlng[1]))
    kmsumList.append(ps.vincenty(coords_1, coords_2).km)
  cdat['km'] = kmsumList
  return cdat


# if __name__ == '__main__':
#     app.debug = True
#     app.run(host = '0.0.0.0', port = 5000)
app.run()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://55cae560a827.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{'type': 'message', 'message': {'type': 'text', 'id': '13988101976999', 'text': 'รสชาติหมาไม่แดก'}, 'timestamp': 1620017946755, 'source': {'type': 'user', 'userId': 'U507c7e6f893486fd572ad089fa47fd91'}, 'replyToken': '1e3620bcd2b2436eaa90ac0550063772', 'mode': 'active'}
class =  neg


/content/drive/Shareddrives/Pretend To Understand NLP/delete_module.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(counts)
[2021-05-03 04:59:09,920] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/pyt

{'type': 'message', 'message': {'type': 'text', 'id': '13988103290409', 'text': 'ได้ของแล้ว ไม่ดีเลยอะ'}, 'timestamp': 1620017965602, 'source': {'type': 'user', 'userId': 'U507c7e6f893486fd572ad089fa47fd91'}, 'replyToken': '5387175aab7c4a06a349b6cffd1fd684', 'mode': 'active'}
class =  neg


...

[2021-05-03 05:10:32,399] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-4-6812fa761dc3>", line 56, in callback
    event_handle(event)
  File "<ipython-inp

{'type': 'message', 'message': {'type': 'text', 'id': '13988151670364', 'text': 'แงงขอโทษ ลืมเพิ่มไม้ยมกก'}, 'timestamp': 1620018645997, 'source': {'type': 'user', 'userId': 'U1cc244f476958589642690d5c58f576d'}, 'replyToken': 'd7da9080b0e34c9f9dea48c4c0dbbd8c', 'mode': 'active'}
class =  neu


[2021-05-03 05:10:47,525] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-4-6812fa761dc3>", line 56, in callback
    event_handle(event)
  File "<ipython-inp

{'type': 'message', 'message': {'type': 'text', 'id': '13988152067870', 'text': 'แย่'}, 'timestamp': 1620018651558, 'source': {'type': 'user', 'userId': 'U1cc244f476958589642690d5c58f576d'}, 'replyToken': 'c0a7f90e077e4024945581b082cdab98', 'mode': 'active'}
class =  neg


/content/drive/Shareddrives/Pretend To Understand NLP/delete_module.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(counts)
[2021-05-03 05:10:53,164] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/pyt